In [56]:
import bisect
import datetime, random
import copy

from traitlets import HasTraits, List, observe


class Segment(object):
    def __init__(self, begin, end, label=None):
        self.begin = begin
        self.end = end
        self.label = label

    @property
    def breakpoints(self):
        return [self.begin, self.end]

    @classmethod
    def from_db(cls, idx, db_path):
        pass

    def __str__(self):
        label = self.label if self.label else 'undefined' 
        s = "Temporal segment\nbegin: {begin}\nend: {end}\nlabel: {label}".format(begin=self.begin,
                                                                                  end=self.end,
                                                                                  label=label)
        return s


class Segmentation(HasTraits):
    segments = List(trait=List())
    breakpoints = List()

    def __init__(self, breakpoints=None, segments=None):
        super(Segmentation, self).__init__()
        self.breakpoints = breakpoints if breakpoints else []
        self.segments = segments if segments else []

    @classmethod
    def from_datelist(cls, dates):
        begin = min(dates)
        end = max(dates)
        instance = cls()
        instance.breakpoints = [begin, end]
        return instance

    def from_db(cls, feature_id, db_path):
        # query to retrieve segments_idx
        segments = [Segment.from_db(idx) for idx in segments_idx]
        breakpoints = self.compute_breakpoints(segments)
        instance = cls(breakpoints=breakpoints, segments=segments)
        return instance

    def add_breakpoints(self, date):
        bp = copy.deepcopy(self.breakpoints)
        bisect.insort(bp, date)
        self.breakpoints = bp

    def remove_breakpoints(self, date):
        bp = copy.deepcopy(self.breakpoints)
        if date in bp:
            bp.remove(date)
        else:
            ValueError('Not a valid breakpoint date')
        self.breakpoints = bp

    def update_marks(self, interface):
        """Given an Interface instance, update its vline attribute"""
        pass

    @staticmethod
    def compute_breakpoints(segments):
        """Compute breakpoints given a list of segments"""
        bp = []
        for seg in segments:
            bp += seg.breakpoints
        return sorted(set(bp))

    @observe('breakpoints')
    def _observe_breakpoints(self, change):
        self.segments = list(zip(self.breakpoints[:-1], self.breakpoints[1:]))
        self.vlines = 


In [6]:
# Generate demo date list
start_datetime = datetime.datetime.now()
datetimes_list = [start_datetime]

for _ in range(9):
    delta_days = random.randint(1, 10)
    new_datetime = datetimes_list[-1] + datetime.timedelta(days=delta_days)
    datetimes_list.append(new_datetime)

In [39]:
seg = Segmentation.from_datelist(datetimes_list)
print(seg.segments)
seg.add_breakpoints(datetime.datetime(2024,2,15))
print(seg.segments)
seg.remove_breakpoints(datetime.datetime(2024,2,15))
print(seg.segments)

[[datetime.datetime(2024, 2, 2, 17, 47, 47, 594792), datetime.datetime(2024, 3, 28, 17, 47, 47, 594792)]]
[[datetime.datetime(2024, 2, 2, 17, 47, 47, 594792), datetime.datetime(2024, 2, 15, 0, 0)], [datetime.datetime(2024, 2, 15, 0, 0), datetime.datetime(2024, 3, 28, 17, 47, 47, 594792)]]
[[datetime.datetime(2024, 2, 2, 17, 47, 47, 594792), datetime.datetime(2024, 3, 28, 17, 47, 47, 594792)]]


[[datetime.datetime(2024, 2, 2, 17, 47, 47, 594792),
  datetime.datetime(2024, 2, 15, 0, 0)],
 [datetime.datetime(2024, 2, 15, 0, 0),
  datetime.datetime(2024, 3, 28, 17, 47, 47, 594792)]]

In [58]:
s = Segment(begin=datetimes_list[3], end=datetimes_list[5])
print(s.breakpoints)

[datetime.datetime(2024, 2, 22, 17, 47, 47, 594792), datetime.datetime(2024, 2, 29, 17, 47, 47, 594792)]


In [60]:
a = [3,2,2,4,5,1]
type(sorted(set(a)))

list